In [1]:
from riptide import *

In [2]:
# Load model
iCardio = cobra.io.load_matlab_model('/home/mjenior/Desktop/iCardio/iRno_HeartModel.mat')

In [3]:
iCardio

Name,iCardio_Rno
Memory address,0x07f3d183c35c0
Number of metabolites,2886
Number of reactions,4183
Number of groups,0
Objective expression,1.0*RCR11017 - 1.0*RCR11017_reverse_8d3a1
Compartments,"c, r, s, m, p, g, l, n"


In [4]:
print('Genes: ' + str(len(iCardio.genes)))

Genes: 1728


In [5]:
iCardio.slim_optimize()

1000.0

In [ ]:
# Read in transcriptomic data




In [ ]:
iCardio_ = riptide.contextualize(iCardio, transcriptome=)